# BERT

## Installs and Imports

In [1]:
pip install transformers[torch]


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers datasets evaluate


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install wikitextparser


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pickle
import re
import wikitextparser as wtp
from typing import Iterator, Optional, Union
from enum import Enum
from nltk.tokenize import sent_tokenize
import bz2
from datasets import Dataset
import re
import tqdm
import numpy as np
import requests
import evaluate
import nltk
from transformers import AutoTokenizer
nltk.download('punkt')

2024-01-15 11:16:28.104669: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 11:16:28.144932: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 11:16:28.144963: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 11:16:28.144986: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 11:16:28.153519: I tensorflow/core/platform/cpu_feature_g

True

In [7]:
class Namespace(Enum):
    OWL = 'http://www.w3.org/2002/07/owl#'
    WIKIPEDIA = 'http://en.wikipedia.org/wiki/'

    PREFIX_TEMPLATE = 'Template:'
    PREFIX_CATEGORY = 'Category:'
    PREFIX_FILE = 'File:'
    PREFIX_IMAGE = 'Image:'
    PREFIX_LIST = 'List_of_'
    PREFIX_LISTS = 'Lists_of_'
    PREFIX_LISTCATEGORY = PREFIX_CATEGORY + PREFIX_LISTS

    DBP_ONTOLOGY = 'http://dbpedia.org/ontology/'
    DBP_RESOURCE = 'http://dbpedia.org/resource/'
    DBP_TEMPLATE = DBP_RESOURCE + PREFIX_TEMPLATE
    DBP_CATEGORY = DBP_RESOURCE + PREFIX_CATEGORY
    DBP_FILE = DBP_RESOURCE + PREFIX_FILE
    DBP_IMAGE = DBP_RESOURCE + PREFIX_IMAGE
    DBP_LIST = DBP_RESOURCE + PREFIX_LIST

    CLG_ONTOLOGY = 'http://caligraph.org/ontology/'
    CLG_RESOURCE = 'http://caligraph.org/resource/'

class RdfClass(Enum):
    OWL_THING = 'http://www.w3.org/2002/07/owl#Thing'
    OWL_CLASS = 'http://www.w3.org/2002/07/owl#Class'
    OWL_NAMED_INDIVIDUAL = 'http://www.w3.org/2002/07/owl#NamedIndividual'
    OWL_OBJECT_PROPERTY = 'http://www.w3.org/2002/07/owl#ObjectProperty'
    OWL_DATATYPE_PROPERTY = 'http://www.w3.org/2002/07/owl#DatatypeProperty'

## Dataset Utils and Preparation

In [8]:
# Own implementation

def extract_abstract(wikipage: wtp.WikiText) -> str:
  parsed_page = wtp.parse(wikipage)
  try:
    result = parsed_page.plain_text(replace_bolds_and_italics=True, replace_wikilinks=False).strip(" '\t\n")
    result = re.sub(r'\n+', '\n', result)
    result = re.sub(r' +', ' ', result)
    result = result.split('==', 1)[0]
    if result.split('\n')[0].startswith('[[File'):
      result = result.split('\n')[1:][0]
    return result
  except(IndexError):

    result = parsed_page.get_sections(level=0)[0].pformat()
    result = result.split('\n\n')[1:]
    result = "\n".join(result)
    return result

In [9]:
def add_special_tokens(text):
    # Define special tokens
    start_token = '[E]'
    end_token = '[/E]'

    # Replace double brackets with special tokens
    text = re.sub(r'\[\[(.*?)\]\]', start_token + r'\1' + end_token, text)

    return text

In [10]:
def name2iri(name: str, prefix: Union[str, Enum]) -> str:
    if name == 'Thing':
        return RdfClass.OWL_THING.value
    prefix = prefix.value if isinstance(prefix, Enum) else prefix
    return prefix + name

def name2resource_iri(name: str) -> str:
    return name2iri(name, Namespace.DBP_RESOURCE)

def label2name(label: str) -> str:
    return label.replace(' ', '_')

In [11]:
def capitalize(text: str) -> str:
    return text[0].upper() + text[1:] if len(text) > 1 else text.upper()

In [12]:
def _remove_language_tag(link_target: str) -> str:
    if not link_target or link_target[0] != ':':
        return link_target
    if len(link_target) < 4 or link_target[3] != ':':
        return link_target[1:]
    return link_target[4:]

In [13]:
def get_resource_name_for_wikilink(wikilink: wtp.WikiLink) -> Optional[str]:
    return label2name(capitalize(_remove_language_tag(wikilink.target.strip())))

In [14]:
triples = pickle.load(open('triples.p', 'rb'))

In [15]:
relations = [
    'http://dbpedia.org/ontology/birthPlace',
    'http://dbpedia.org/ontology/family',
    'http://dbpedia.org/ontology/deathPlace',
    'http://dbpedia.org/ontology/producer',
    'http://dbpedia.org/ontology/writer',
    'http://dbpedia.org/ontology/subsequentWork',
    'http://dbpedia.org/ontology/previousWork',
    'http://dbpedia.org/ontology/artist',
    'http://dbpedia.org/ontology/formerTeam'  
]

In [14]:
data = pickle.load(open('dataPickle.p', 'rb'))

In [14]:
data = pickle.load(open('/pfs/work7/workspace/scratch/ma_nfuerhau-masterthesis/dataPickle.p', 'rb'))

In [115]:
people = {}

In [72]:
species = {}

In [24]:
works = {}

In [ ]:
agents = {}

In [15]:
musical_works = {}

In [48]:
athletes = {}

In [55]:
types = pickle.load(open('decodedTypes.p', 'rb'))

In [57]:
work_types = set()
for k, v in tqdm.tqdm(types.items()):
    if 'http://dbpedia.org/ontology/Work' in v:
        work_types.add(k)

100%|██████████| 6556328/6556328 [00:02<00:00, 3110199.10it/s]


In [16]:
abstracts = pickle.load(open('abstracts400k.p', 'rb'))

In [ ]:
count = 0
desired = 50000
for k, v in tqdm.tqdm(data.items()):
    if count < desired:
        try:
            if k in birthplace_set:
                people[k] = extract_abstract(v)
                count += 1
        except TypeError:
            continue

In [ ]:
pickle.dump(people, open('data/people50k.p', 'wb'))

In [14]:
species = pickle.load(open('data/species20000.p', 'rb'))

In [ ]:
pickle.dump(abstracts, open('/pfs/work7/workspace/scratch/ma_nfuerhau-masterthesis/abstractsAll.p', 'wb'))

In [21]:
abstracts = pickle.load(open('abstracts.p', 'rb'))

In [18]:
for k, v in tqdm.tqdm(abstracts.items()):
    try:
        if 'https://dbpedia.org/ontology/Work' in types.get(k):
            works[k] = v
    except TypeError:
        continue

100%|██████████| 21309/21309 [00:00<00:00, 895151.72it/s]


In [22]:
people = pickle.load(open('people.p', 'rb'))

In [17]:
triples_birthplace = []

for triple in tqdm.tqdm(triples):
    if triple[1] == relations[0]:
        triples_birthplace.append(triple)

100%|██████████| 19808136/19808136 [00:05<00:00, 3488704.45it/s]


In [31]:
birthplace_set = set()

for triple in tqdm.tqdm(triples):
    if triple[1] == relations[0]:
        birthplace_set.add(triple[0])

100%|██████████| 19808136/19808136 [00:06<00:00, 3012765.39it/s]


In [55]:
triples_deathplace = []

for triple in tqdm.tqdm(triples):
    if triple[1] == relations[2]:
        triples_deathplace.append(triple)

100%|██████████| 19808136/19808136 [00:06<00:00, 3293650.17it/s]


In [70]:
triples_producer = []

for triple in tqdm.tqdm(triples):
    if triple[1] == relations[3]:
        triples_producer.append(triple)

100%|██████████| 19808136/19808136 [00:05<00:00, 3335525.70it/s]


In [78]:
triples_writer = []

for triple in tqdm.tqdm(triples):
    if triple[1] == relations[4]:
        triples_writer.append(triple)

100%|██████████| 19808136/19808136 [00:06<00:00, 2852648.27it/s]


In [85]:
triples_subsequent_work = []

for triple in tqdm.tqdm(triples):
    if triple[1] == relations[5]:
        triples_subsequent_work.append(triple)

100%|██████████| 19808136/19808136 [00:05<00:00, 3326417.19it/s]


In [93]:
triples_previous_work = []

for triple in tqdm.tqdm(triples):
    if triple[1] == relations[6]:
        triples_previous_work.append(triple)

100%|██████████| 19808136/19808136 [00:06<00:00, 3271105.02it/s]


In [101]:
triples_artist = []

for triple in tqdm.tqdm(triples):
    if triple[1] == relations[7]:
        triples_artist.append(triple)

100%|██████████| 19808136/19808136 [00:05<00:00, 3312182.73it/s]


In [109]:
triples_former_team = []

for triple in tqdm.tqdm(triples):
    if triple[1] == relations[8]:
        triples_former_team.append(triple)

100%|██████████| 19808136/19808136 [00:05<00:00, 3312424.39it/s]


In [44]:
triples_family = []

for triple in tqdm.tqdm(triples):
    if triple[1] == relations[1]:
        triples_family.append(triple)

100%|██████████| 19808136/19808136 [00:06<00:00, 3246186.19it/s]


In [ ]:
triples_birthplace_domains = set()

for triple in triples_birthplace:
    triples_birthplace_domains.add(triple[0])

## Create Datasets

In [110]:
triples_dict = {}

for item in tqdm.tqdm(triples_former_team):
    key = item[0]
    value = (item[1], item[2])

    if key in triples_dict:
        triples_dict[key] += (value,)
    else:
        triples_dict[key] = (value,)

100%|██████████| 129947/129947 [00:00<00:00, 737031.79it/s]


In [19]:
# method to create dataset out of abstracts and put special tokens about entities

def replace_entities_one_at_a_time_dict(data, relation, triples_dict):

    modified_abstracts = {'text': [],
                          'label': []}

    for resource, abstract in tqdm.tqdm(data.items()):

        
        # Regular expression pattern to match entities within double brackets
        entity_pattern = r'\[\[(.*?)\]\]'
    
        # Find all matches of the entity pattern in the text
        entity_matches = re.findall(entity_pattern, abstract)
    
        links = wtp.parse(abstract).wikilinks

        if len(links) > 0 and len(links) == len(entity_matches):
    
            for index, entity in enumerate(entity_matches):
        
                domain = resource
        
                range = name2resource_iri(get_resource_name_for_wikilink(links[index]))
                modified_text = abstract.replace(f'[[{entity}]]', f'[E]{entity}[/E]')

                # Remove double brackets for all other entities
                modified_text = re.sub(entity_pattern, r'\1', modified_text)
                
                if domain in triples_dict:
                    for triple in triples_dict.get(domain, None):
                        if triple[0] == relation and triple[1] == range:
                            modified_abstracts['label'].append(1)
                            modified_abstracts['text'].append(modified_text)
                        else:
                            modified_abstracts['label'].append(0)
                            modified_abstracts['text'].append(modified_text)
        
    ds = Dataset.from_dict(modified_abstracts)

    return ds

In [78]:
species10000 = {}
for k, v in list(species.items())[10000:20000]:
    species10000[k] = v

In [ ]:
ds = replace_entities_one_at_a_time(people, relations[0], triples)

In [111]:
ds = replace_entities_one_at_a_time_dict(abstracts, relations[8], triples_dict)

100%|██████████| 400000/400000 [04:28<00:00, 1491.75it/s]


In [32]:
ds = pickle.load(open('data/datasets/former_team_ds.p', 'rb'))

In [54]:
tokens = pickle.load(open('data/datasets/tokenizedDeathplace.p', 'rb'))

In [55]:
train = list(tokens['train']['label'])
test = list(list(tokens['test']['label']))
labels = train + test

In [56]:
positive = 0
negative = 0
for label in labels:
    if label == 1:
        positive += 1
    else:
        negative += 1

print(positive, negative)

352 35000


In [32]:
zero_label = ds.filter(lambda example: example["label"] == 0)
one_label = ds.filter(lambda example: example["label"] == 1)

Filter:   0%|          | 0/107460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/107460 [00:00<?, ? examples/s]

In [33]:
shuffled_50k_zero_label = zero_label.shuffle(seed=42).select(range(50000))

In [34]:
from datasets import concatenate_datasets
conc_ds = concatenate_datasets([shuffled_50k_zero_label, one_label])

In [35]:
ds = conc_ds.shuffle(seed=42)

In [36]:
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 53519
})

In [91]:
pickle.dump(ds, open('data/datasets/species10000_ds.p', 'wb'))

In [16]:
ds = pickle.load(open('data/datasets/species_ds.p', 'rb'))

In [21]:
producer_ds = ds.train_test_split(test_size=0.2)

In [24]:
ds = pickle.load(open('data/datasets/artist_ds.p', 'rb'))

In [36]:
ds[630:632]

{'text': ['"Take Me Out to the Ball Game" is a 1908 Tin Pan Alley song by [E]Jack Norworth[/E] and Albert Von Tilzer which has become the unofficial anthem of North American baseball, although neither of its authors had attended a game before writing the song. The song\'s chorus is traditionally sung as part of the seventh-inning stretch of a baseball game. Fans are generally encouraged to sing along, and at some ballparks, the words "home team" are replaced with the team name.\n',
  '"Take Me Out to the Ball Game" is a 1908 Tin Pan Alley song by Jack Norworth and [E]Albert Von Tilzer[/E] which has become the unofficial anthem of North American baseball, although neither of its authors had attended a game before writing the song. The song\'s chorus is traditionally sung as part of the seventh-inning stretch of a baseball game. Fans are generally encouraged to sing along, and at some ballparks, the words "home team" are replaced with the team name.\n'],
 'label': [0, 0]}

In [114]:
from datasets import DatasetDict

ds_train_devtest = ds.train_test_split(test_size=0.4, seed=42)
ds_devtest = ds_train_devtest['test'].train_test_split(test_size=0.5, seed=42)

former_team_ds = DatasetDict({
    'train': ds_train_devtest['train'],
    'val': ds_devtest['train'],
    'test': ds_devtest['test']
})

In [115]:
former_team_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 23488
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 7829
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7830
    })
})

In [116]:
pickle.dump(former_team_ds, open('models/finished_models/data_used/former_team_ds', 'wb'))

## Preprocess

In [35]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

custom_tokens = ["[E]", "[/E]"]

tokenizer.add_tokens(custom_tokens)

2

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_deathplace = deathplace_ds.map(preprocess_function, batched=True)

In [38]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

In [39]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")
recall = evaluate.load("recall")

In [40]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    return clf_metrics.compute(predictions=predictions, references=labels)

## Train

In [41]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [42]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.resize_token_embeddings(len(tokenizer))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30524, 768)

In [102]:
pickle.dump(tokenized_family, open('data/datasets/tokenizedFamily.p', 'wb'))

In [12]:
tokenized_species = pickle.load(open('data/datasets/tokenizedSpecies.p', 'rb'))

In [69]:
training_args = TrainingArguments(
    output_dir="models/finished_models/deathplace3",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_deathplace["train"],
    eval_dataset=tokenized_deathplace["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/opt/bwhpc/common/jupyter/tensorflow/2023-10-10/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.141600,0.115198,0.944718,0.691760,0.652591,0.735931
2,0.107900,0.118162,0.941252,0.707539,0.609546,0.843074
3,0.086600,0.131670,0.945630,0.706981,0.647748,0.778139
4,0.069000,0.156039,0.943167,0.700625,0.630078,0.788961
5,0.056700,0.181543,0.944809,0.697953,0.647822,0.756494


/opt/bwhpc/common/jupyter/tensorflow/2023-10-10/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/bwhpc/common/jupyter/tensorflow/2023-10-10/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/bwhpc/common/jupyter/tensorflow/2023-10-10/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/bwhpc/common/jupyter/tensorflow/2023-10-10/lib/python3.9/site-pac

TrainOutput(global_step=2570, training_loss=0.09142308791787708, metrics={'train_runtime': 1585.9644, 'train_samples_per_second': 103.672, 'train_steps_per_second': 1.62, 'total_flos': 4.32607197222912e+16, 'train_loss': 0.09142308791787708, 'epoch': 5.0})

## Inference

In [113]:
positive = '''
Joseph Robinette Biden Jr. (born November 20, 1942) is an American politician who 
is the 46th and current president of the United States. Ideologically a moderate member 
of the Democratic Party, he previously served as the 47th vice president from 2009 to 2017 
under President Barack Obama and represented Delaware in the United States Senate from 1973 to 2009. 
Born in [E]Scranton[/E], Pennsylvania, Biden moved with his family to Delaware in 1953.
'''

In [115]:
negative = '''
Joseph Robinette Biden Jr. (born November 20, 1942) is an American politician who 
is the 46th and current president of the United States. Ideologically a moderate member 
of the Democratic Party, he previously served as the 47th vice president from 2009 to 2017 
under President Barack Obama and represented Delaware in the United States Senate from 1973 to 2009. 
Born in Scranton, Pennsylvania, Biden moved with his family to [E]Delaware[/E] in 1953.
'''

In [117]:
positive = '''
Angela Dorothea Merkel (born 17 July 1954) is a German former politician and scientist who 
served as chancellor of Germany from 2005 to 2021. A member of the Christian Democratic Union (CDU), 
she previously served as Leader of the Opposition from 2002 to 2005 and as Leader of the Christian Democratic 
Union from 2000 to 2018.[9] Merkel was the first female chancellor of Germany.[b] During her chancellorship, 
Merkel was frequently referred to as the de facto leader of the European Union (EU) and the most powerful woman 
in the world. Beginning in 2016, she was often described as the leader of the free world.[13]
Merkel was born in [E]Hamburg[/E] in then-West Germany. Her family moved to East Germany when she was an infant. 
Merkel obtained a doctorate in quantum chemistry in 1986 and worked as a research scientist until 1989.[14] 
She then entered politics in the wake of the Revolutions of 1989, briefly serving as deputy spokeswoman for 
the first democratically elected government of East Germany led by Lothar de Maizière. 
Following German reunification in 1990, Merkel was elected to the Bundestag for the state of Mecklenburg-Vorpommern. 
As the protégée of chancellor Helmut Kohl, Merkel was appointed as Minister for Women and Youth in 1991, later 
becoming Minister for the Environment, Nature Conservation and Nuclear Safety in 1994. After the CDU lost the 1998 
federal election, Merkel was elected general secretary of the party. She then became the party's first female 
leader, and the first female Leader of the Opposition two years later.
'''

In [123]:
negative = '''
Angela Dorothea Merkel (born 17 July 1954) is a German former politician and scientist who 
served as chancellor of Germany from 2005 to 2021. A member of the Christian Democratic Union (CDU), 
she previously served as Leader of the Opposition from 2002 to 2005 and as Leader of the Christian Democratic 
Union from 2000 to 2018.[9] Merkel was the first female chancellor of Germany.[b] During her chancellorship, 
Merkel was frequently referred to as the de facto leader of the European Union (EU) and the most powerful woman 
in the world. Beginning in 2016, she was often described as the leader of the free world.[13]
Merkel was born in Hamburg in then-West Germany. Her family moved to East Germany when she was an infant. 
Merkel obtained a doctorate in quantum chemistry in 1986 and worked as a research scientist until 1989.[14] 
She then entered politics in the wake of the Revolutions of 1989, briefly serving as deputy spokeswoman for 
the first democratically elected government of East Germany led by Lothar de Maizière. 
Following German reunification in 1990, Merkel was elected to the Bundestag for the state of [E]Berlin[/E]. 
As the protégée of chancellor Helmut Kohl, Merkel was appointed as Minister for Women and Youth in 1991, later 
becoming Minister for the Environment, Nature Conservation and Nuclear Safety in 1994. After the CDU lost the 1998 
federal election, Merkel was elected general secretary of the party. She then became the party's first female 
leader, and the first female Leader of the Opposition two years later.
'''

In [20]:
positive = '''
An oak is a hardwood tree or shrub in the genus Quercus of the [E]beech[/E] family. 
They have spirally arranged leaves, often with lobed edges, and a nut called an acorn, borne in a cup. 
The genus is widely distributed in the Northern Hemisphere; it includes some 500 species, both deciduous and evergreen. 
Fossil oaks date back to the Middle Eocene. Molecular phylogeny shows that the genus is divided into Old World and New World 
clades, but many oak species hybridise freely, making the genus's history difficult to resolve.
'''

In [21]:
negative = '''
An oak is a hardwood tree or shrub in the genus Quercus of the beech family. 
They have spirally arranged leaves, often with lobed edges, and a nut called an [E]acorn[/E], borne in a cup. 
The genus is widely distributed in the Northern Hemisphere; it includes some 500 species, both deciduous and evergreen. 
Fossil oaks date back to the Middle Eocene. Molecular phylogeny shows that the genus is divided into Old World and New World 
clades, but many oak species hybridise freely, making the genus's history difficult to resolve.
'''

In [106]:
positive = '''
Zebras (subgenus Hippotigris) are African equines with distinctive 
black-and-white striped coats. There are three living species: the Grévy's zebra (Equus grevyi), plains zebra (E. quagga), 
and the mountain zebra (E. zebra). Zebras share the genus Equus with horses and asses, the three groups being the only 
living members of the family [E]Equidae[/E]. Zebra stripes come in different patterns, unique to each individual. 
Several theories have been proposed for the function of these stripes, with most evidence supporting them as a deterrent 
for biting flies. Zebras inhabit eastern and southern Africa and can be found in a variety of habitats such as savannahs, 
grasslands, woodlands, shrublands, and mountainous areas.
'''

In [107]:
negative = '''
Zebras (subgenus Hippotigris) are African equines with distinctive 
black-and-white striped coats. There are three living species: the Grévy's zebra (Equus grevyi), plains zebra (E. quagga), 
and the mountain zebra (E. zebra). Zebras share the genus Equus with horses and [E]asses[/E], the three groups being the only 
living members of the family Equidae. Zebra stripes come in different patterns, unique to each individual. 
Several theories have been proposed for the function of these stripes, with most evidence supporting them as a deterrent 
for biting flies. Zebras inhabit eastern and southern Africa and can be found in a variety of habitats such as savannahs, 
grasslands, woodlands, shrublands, and mountainous areas.
'''

In [108]:
from transformers import pipeline

classifier = pipeline("text-classification", model="models/family/checkpoint-2572")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [109]:
classifier(positive)

[{'label': 'POSITIVE', 'score': 0.873722493648529}]

In [110]:
classifier(negative)

[{'label': 'NEGATIVE', 'score': 0.9994033575057983}]

## Testing

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

In [6]:
def compute_metrics_eval(p):
    predictions, labels = p.predictions, p.label_ids
    
    # Apply threshold to predictions
    predictions = (predictions[:, 1] > 0.4).astype(int)
    
    # Accuracy
    accuracy = accuracy_score(labels, predictions)

    # Precision, Recall, F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')

    # Classification Report
    class_report = classification_report(labels, predictions)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'classification_report': class_report,
    }


In [7]:
new_test_models = [
    'models/finished_models/birthplace3/checkpoint-2740',
    'models/finished_models/family3/checkpoint-475',
    'models/finished_models/deathplace3/checkpoint-2570',
    'models/finished_models/producer2/checkpoint-3055',
    'models/finished_models/artist2/checkpoint-840',
    'models/finished_models/previous_work_2/checkpoint-1370',
    'models/finished_models/subsequent_work_2/checkpoint-1850',
    'models/finished_models/writer4/checkpoint-2935',
    'models/finished_models/former_team2/checkpoint-3670'
]

In [8]:
old_test_models = [
    'models/birthplace/checkpoint-4980',
    'models/family/checkpoint-2572',
    'models/deathplace/checkpoint-9950',
    'models/producer/checkpoint-8328',
    'models/artist/checkpoint-4656',
    'models/previous_work/checkpoint-6196',
    'models/subsequent_work2/checkpoint-4564',
    'models/writer3/checkpoint-6846',
    'models/formerTeam/checkpoint-5376'    
]

In [9]:
test_data = [
    'models/finished_models/data_used/birthplace_ds.p',
    'models/finished_models/data_used/family_ds.p',
    'models/finished_models/data_used/deathplace_ds.p',
    'models/finished_models/data_used/producer_ds.p',
    'models/finished_models/data_used/artist_ds.p',
    'models/finished_models/data_used/previous_work_ds.p',
    'models/finished_models/data_used/subsequent_work_ds.p',
    'models/finished_models/data_used/writer_ds.p',
    'models/finished_models/data_used/former_team_ds.p'
]

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [12]:
single_threshold_models = [
    'models/birthplace/checkpoint-4980',
    'models/family/checkpoint-2572',
    'models/deathplace/checkpoint-9950',
    'models/producer/checkpoint-8328',
    'models/artist/checkpoint-4656',
    'models/previous_work/checkpoint-6196',
    'models/finished_models/subsequent_work_2/checkpoint-1850',
    'models/writer3/checkpoint-6846',
    'models/formerTeam/checkpoint-5376' 
]

In [ ]:
varying_thresholds_models = [
    'models/finished_models/birthplace3/checkpoint-2740',
    'models/finished_models/family3/checkpoint-475',
    'models/finished_models/deathplace3/checkpoint-2570',
    'models/finished_models/producer2/checkpoint-3055',
    'models/finished_models/artist2/checkpoint-840',
    'models/finished_models/previous_work_2/checkpoint-1370',
    'models/finished_models/subsequent_work_2/checkpoint-1850',
    'models/finished_models/writer4/checkpoint-2935',
    'models/finished_models/former_team2/checkpoint-3670'
]

In [13]:
test_results = {}

def compute_metrics_eval(p):
    predictions, labels = p.predictions, p.label_ids
    
    # Apply threshold to predictions
    binary_predictions = (predictions[:, 1] > 0.5).astype(int)
    
    # Accuracy
    accuracy = accuracy_score(labels, binary_predictions)

    # Precision, Recall, F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, binary_predictions, average='binary') 

    # Classification Report
    class_report = classification_report(labels, binary_predictions)

    probabilities = torch.nn.functional.softmax(torch.from_numpy(predictions), dim=-1)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'classification_report': class_report,
        'probabilities': probabilities,
        'labels': labels
    }

for index, test_model in enumerate(single_threshold_models):
    model = AutoModelForSequenceClassification.from_pretrained(test_model)
    tokenizer = AutoTokenizer.from_pretrained(test_model)

    custom_tokens = ["[E]", "[/E]"]
    tokenizer.add_tokens(custom_tokens)

    name = test_data[index].removeprefix('models/finished_models/data_used/').removesuffix('_ds.p')

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Set up the Trainer for evaluation
    eval_args = TrainingArguments(
        output_dir=f"models/finished_models/evaluation/{name}_eval",
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
    )
    
    eval_trainer = Trainer(
        model=model,
        args=eval_args,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_eval,
    )

    tokenized = pickle.load(open(test_data[index], 'rb')).map(preprocess_function, batched=True)
    
    results = eval_trainer.evaluate(tokenized["test"])
    
    test_results[name] = results

    print(name)
    print('Precision', results['eval_precision'])
    print('Recall', results['eval_recall'])
    print('F1', results['eval_f1'])
    print()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Map:   0%|          | 0/35059 [00:00<?, ? examples/s]

Map:   0%|          | 0/11687 [00:00<?, ? examples/s]

Map:   0%|          | 0/11687 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

Map:   0%|          | 0/7829 [00:00<?, ? examples/s]

Map:   0%|          | 0/7830 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "              precision    recall  f1-score   support

           0       1.00      0.80      0.89      7461
           1       0.19      0.96      0.32       369

    accuracy                           0.81      7830
   macro avg       0.59      0.88      0.60      7830
weighted avg       0.96      0.81      0.86      7830
" of type <class 'str'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "tensor([[9.9999e-01, 1.3996e-05],
        [9.9997e-01, 3.2242e-05],
        [9.9992e-01, 8.2653e-05],
        ...,
        [4.3285e-03, 9.9567e-01],
        [9.9985e-01, 1.4949e-04],
        [9.9995e-01, 5.3268e-05]])" of type <class 'torch.Tensor'> for key "eval/probabilities" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a val

former_team
Precision 0.19168466522678185
Recall 0.962059620596206
F1 0.31967582170193604



In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [16]:
for k, v in test_results.items():
    probabilities = v['eval_probabilities']
    labels = v['eval_labels']

    best_precision = 0
    best_threshold = 0

    for threshold in range(1, 1000):
        threshold /= 1000
    
        predictions = (probabilities[:, 1] >= threshold).numpy().astype(int)
    
        precision = precision_score(labels, predictions, zero_division=0)
    
        # Track the best precision and threshold
        if precision >= best_precision and precision <= 0.95:
          best_precision = precision
          best_threshold = threshold

    predictions = (probabilities[:, 1] >= best_threshold).numpy().astype(int)

    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    f1 = f1_score(labels, predictions)

    print(k)
    print('Threshold', best_threshold)
    print('Precision', precision)
    print('Recall', recall)
    print('F1', f1)
    print()

birthplace
Threshold 0.985
Precision 0.9478260869565217
Recall 0.1290704558910598
F1 0.22720166753517457

family
Threshold 0.706
Precision 0.6666666666666666
Recall 0.02702702702702703
F1 0.051948051948051945

deathplace
Threshold 0.986
Precision 0.9333333333333333
Recall 0.014846235418875928
F1 0.029227557411273485

producer
Threshold 0.948
Precision 0.75
Recall 0.021897810218978103
F1 0.04255319148936171

artist
Threshold 0.924
Precision 0.9130434782608695
Recall 0.19626168224299065
F1 0.32307692307692304

previous_work
Threshold 0.982
Precision 0.8235294117647058
Recall 0.175
F1 0.28865979381443296

subsequent_work
Threshold 0.989
Precision 0.8
Recall 0.056338028169014086
F1 0.10526315789473684

writer
Threshold 0.99
Precision 0.859375
Recall 0.17973856209150327
F1 0.2972972972972973

former_team
Threshold 0.607
Precision 0.7777777777777778
Recall 0.018970189701897018
F1 0.037037037037037035



In [59]:
# Load the model from the directory
model = AutoModelForSequenceClassification.from_pretrained("models/birthplace2/checkpoint-2740")
tokenizer = AutoTokenizer.from_pretrained("models/birthplace2/checkpoint-2740")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [92]:
# Set up the Trainer for evaluation
eval_args = TrainingArguments(
    output_dir="models/birthplace2_eval",
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
)
eval_trainer = Trainer(
    model=model,
    args=eval_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_eval,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [93]:
# Evaluate on the test set with a different threshold
test_results = eval_trainer.evaluate(tokenized_birthplace["test"])

Trainer is attempting to log a value of "              precision    recall  f1-score   support

           0       0.97      0.98      0.98      9998
           1       0.87      0.84      0.86      1689

    accuracy                           0.96     11687
   macro avg       0.92      0.91      0.92     11687
weighted avg       0.96      0.96      0.96     11687
" of type <class 'str'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
